In [1]:
from cloudant.client import Cloudant
from cloudant.query import Query
from cloudant.database import CloudantDatabase
import requests
import json
from re import match
import pandas as pd
import time
import ast,yaml,requests

In [ ]:
def connectCloudant(cloudantFile):
    '''
    Connects to cloudant and returns the connection client
    :param cloudantFile: str, Path to file with Cloudant access credentials
    :return: Cloudant client instance
    '''

    with open(cloudantFile) as outfile:
        authDetails = json.load(outfile)
        outfile.close()

    username = authDetails["username"]
    password = authDetails["password"]
    accountName = authDetails["account"]

    # Connect to DB
    
    # https://stackoverflow.com/questions/34837026/whats-the-meaning-of-pool-connections-in-requests-adapters-httpadapter
    httpAdapter = requests.adapters.HTTPAdapter(pool_connections=15, pool_maxsize=100)
    client = Cloudant(username, password, account=accountName, connect=True, auto_renew=True,adapter=httpAdapter)

    return client


def getDBNames(client, reqDB):
    '''
    Fetches the name of the required DB to connect to
    :param client: CloudantInstance, Cloudant client instance
    :param reqDB: str/regex, The DB nae to be fetched
    :return: The Cloudant DB instance
    '''

    allDB = client.all_dbs()
    dbNames = []
    for db in allDB:
        if match(reqDB, db):
            dbNames.append(db)
    return dbNames

def getCCData(client,dbNames,designDocID,viewName,currentStartkey,currentEndKey,rowPerPage,skip):
    
    docList = []
    for db in dbNames:
        docList = client[db].get_view_result(ddoc_id= str(designDocID), view_name= str(viewName), raw_result= True, include_docs= True,startkey = currentStartkey,endkey = currentEndKey,limit = rowPerPage, skip = skip)
    return docList
    
def convertToCSV(docList):
    
    purchaseDocsDF = pd.DataFrame(columns= ["IATACode","FlightNumber","Date","Origin", "Destination","EmployeeID", "TabletID",
                                                "PurchaseID", "PaymentTime", "Turnover", "Currency","Completed","Quantity","Price","Catelog","Category","Complementary","Discounted","ProductID","Short_Description","Refunded","SyncTime","Last_Payment_Status_Updated","Payment_Status"])
    i=0     
    for d in docList['rows']:
        iataCode = d['doc']['airline_iata_code']
        flightNmber = d['doc']["flight_leg_identifier"]["flight_number"]
        date = d['doc']["flight_leg_identifier"]["date"]
        origin = d['doc']["flight_leg_identifier"]["origin"]
        destination = d['doc']["flight_leg_identifier"]["destination"]

        employeeID = d['doc']["employee_identifier"]
        tabletID = d['doc']["tablet_id"]
        purchaseID = d['doc']["order_identifier"]
        paymentTime = d['doc']["last_payment_status_update"]
        turnover = d['doc']["total_purchase_value"]
        currency = d['doc']["currency_code"]
        completed = d['doc']["completed"]

        quantity= d["doc"]["line_items"][0]["quantity"]
        price = d["doc"]["line_items"][0]["product"]["price"]
        catelog = d["doc"]["line_items"][0]['product']["catalog"]
        category = d["doc"]["line_items"][0]['product']["category"]
        complementary = d["doc"]["line_items"][0]["product"]["complementary_allowed"]
        discounted= d["doc"]["line_items"][0]["product"]["discounted"]
        product_id= d["doc"]["line_items"][0]["product"]["product_id"]
        short_description = d["doc"]["line_items"][0]["product"]["short_description"]
        refunded = d["doc"]["refunded"]
        synctime = d["doc"]['sync_time']
        last_payment_status_updated = d["doc"]["last_payment_status_update"]
        payment_status = d["doc"]["payment_status"]
        purchaseDocsDF.loc[i, :] = [iataCode,flightNmber,date,origin,destination,employeeID,tabletID,purchaseID,paymentTime,turnover,currency,completed,quantity,price,catelog,category,complementary,discounted,product_id,short_description,refunded,synctime,last_payment_status_updated,payment_status]
        i += 1

    print("Completed") 
    return purchaseDocsDF

def mainFuncGetCCData(docType, reqDB, dbType, iataCode):
    
    if dbType == "Production":
        cloudantFileName = "CCProductionCloudantAuthorization.json"
    
    client = connectCloudant(cloudantFileName)
    
    dbNames = getDBNames(client,reqDB)
    dbNamesFileName = "DBNames.csv"
    dbNamesDF = pd.DataFrame(dbNames,columns= ["DBNames"])
    dbNamesDF.to_csv(dbNamesFileName)
    
    start = time.clock()
    hasRow = True
    rowPerPage = 5000
    page = 0
    currentStartkey= ["HV","2017-12-01"]
    currentEndKey = ["HV","2018-03-31"]
    designDocID= "_design/Purchase"
    viewName = "by_date_range"
    skip = 0
    
    start = time.clock()
    print("Start_Time",start)
    
   
    while hasRow :
        page = page + 1
        print("-- Page %s --" % (page))
        docList = getCCData(client,dbNames,designDocID,viewName,currentStartkey,currentEndKey,rowPerPage,skip)
        
        print skip
        skip+=rowPerPage
        print len(docList['rows'])
        if len(docList['rows']) == 0:
            hasRow = False
        total_rows = docList['total_rows']
        if total_rows < skip:
            hasRow = False
            break
        
        docFile = "CSV_Purchase_HV_Pagination_Code.csv"
        if len(docList['rows']) == 0:
            hasRow = False 
        else:
            with open(docFile, "a") as outfile:
                hv_data = convertToCSV(docList)
                hv_data.to_csv(outfile,sep = ",",encoding="utf-8", mode="a", header= False, index= False)

    stop = time.clock()
    print("Stop_Time",stop)
    print("Time for view query - " + str(stop - start))


In [ ]:
if __name__ == "__main__":
    mainFuncGetCCData("Purchase", "^mi-master$", "Production", "HV")

('Start_Time', 2.143978)
-- Page 1 --
0
5000
Completed
-- Page 2 --
5000
5000
Completed
-- Page 3 --
10000
5000
Completed
-- Page 4 --
15000
5000
Completed
-- Page 5 --
20000
5000
Completed
-- Page 6 --
25000
5000
Completed
-- Page 7 --
30000
5000
Completed
-- Page 8 --
35000
5000
Completed
-- Page 9 --
40000
5000
Completed
-- Page 10 --
45000
5000
Completed
-- Page 11 --
50000
5000
Completed
-- Page 12 --
55000
5000
Completed
-- Page 13 --
60000
5000
Completed
-- Page 14 --
65000
5000
Completed
-- Page 15 --
70000
5000
Completed
-- Page 16 --
75000
5000
Completed
-- Page 17 --
80000
5000
Completed
-- Page 18 --
